In [1]:
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17571441837140735530
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6946941824
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17873416760617332825
physical_device_desc: "device: 0, name: Quadro RTX 4000, pci bus id: 0000:b3:00.0, compute capability: 7.5"
]


In [1]:
#Evaluate a model (saved during training in the 'models' directory) 
#through Monte Carlo analysis over the random parameter space for the performance indicators 
#'Successful Landing', 'Collection Ratio', 'Collection Ratio and Landed' as defined in the paper 
#(plus 'Boundary Counter' counting safety controller activations), e.g. for 1000 Monte Carlo iterations:

import argparse
import os

import numpy as np

import import_ipynb
from MainEnvironment_test3 import EnvironmentParams, Environment
from Utils_test3 import read_config

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf


def eval_logs(event_path):
    event_acc = EventAccumulator(event_path, size_guidance={'tensors': 100000})
    event_acc.Reload()

    _, _, vals = zip(*event_acc.Tensors('successful_landing'))
    has_landed = [tf.make_ndarray(val) for val in vals]

    ###
    _, _, vals = zip(*event_acc.Tensors('boundary_counter'))
    boundary_counter = [tf.make_ndarray(val) for val in vals]
    
    _, _, vals = zip(*event_acc.Tensors('movement_ratio'))
    charging_attempts = [tf.make_ndarray(val) for val in vals]
    
    ###
    _, _, vals = zip(*event_acc.Tensors('current_movement_budget'))
    current_movement_budget = [tf.make_ndarray(val) for val in vals]
    
    _, _, vals = zip(*event_acc.Tensors('tour_counter'))
    tour_counter = [tf.make_ndarray(val) for val in vals]    
    
    _, _, vals = zip(*event_acc.Tensors('tough_counter'))
    tough_counter = [tf.make_ndarray(val) for val in vals]     

#    print("Successful Landing:", sum(has_landed) / len(has_landed))

#    print("Successful Communicating:", sum(successful_communicating) / len(successful_communicating))
#    print("Collection ratio:", sum(cr) / len(cr))
#    print("Collection ratio and landed:", sum(crac) / len(crac))
    print("Boundary counter:", sum(boundary_counter) / len(boundary_counter))
    print("Tour counter:", sum(tour_counter) / len(tour_counter))
    print("Tough counter:", sum(tough_counter) / len(tough_counter))
    print("Successful Landing:", sum(has_landed) / len(has_landed))
    print("Current budget:", sum(current_movement_budget) / len(current_movement_budget))


def mc(args, params: EnvironmentParams):
    if args.num_agents is not None:
        num_range = [int(i) for i in args.num_agents]
        params.grid_params.num_agents_range = num_range
        
    if args.starts is not None:
        start_end = [int(i) for i in args.starts]
        params.grid_params.start_end_area = start_end       
        
    if args.energys is not None:
        energy = [int(i) for i in args.energys]
        params.grid_params.movement_range = energy      
        
    if args.toughs is not None:
        tough = [i for i in args.toughs]
        params.reward_params.tough_penalty = tough         
      
    #############################################################    
    if args.tours is not None:
        tour = [int(i) for i in args.tours]
        params.grid_params.tour_count_range = tour 
        
    try:
        env = Environment(params)
        env.agent.load_weights(args.weights)

        env.eval(int(args.samples), show=args.show)
    except AttributeError:
        print("Not overriding log dir, eval existing:")

    eval_logs("logs/training/" + args.id + "/test")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', default='models/grid_image_best', help='Path to weights')
    parser.add_argument('--config', default='config/grid_image.json', help='Config file for agent shaping')
    parser.add_argument('--id', default = 'grid_image', help='Id for exported files')
    parser.add_argument('--samples', default = 300, help='Id for exported files')
    parser.add_argument('--seed', default=0, help="Seed for repeatability")
    parser.add_argument('--show', default=False, help="Show individual plots, allows saving")
    parser.add_argument('--num_agents', default=[1, 1], help='Overrides num agents range, argument 12 for range [1,2]')
    ###########################################################################################################################
    parser.add_argument('--starts', default=[1, 1], help='')#2, 1
    parser.add_argument('--energys', default=[180, 180], help='')#100
    parser.add_argument('--toughs', default=[0.05, 0.05], help='')#0.25       
    parser.add_argument('--tours', default=[26, 26, 26, 26], help='') 
   #parser.add_argument('--tours', default=[0.9, 0.9, 0.9], help='') 
    

    #args = parser.parse_args()
    args = parser.parse_args(args=[])

    if args.seed:
        np.random.seed(int(args.seed))

    params = read_config(args.config)

    if args.id is not None:
        params.model_stats_params.save_model = "models/" + args.id
        params.model_stats_params.log_file_name = args.id

    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    mc(args, params)

importing Jupyter notebook from MainEnvironment_test3.ipynb
importing Jupyter notebook from Agent_test3.ipynb
importing Jupyter notebook from Trainer_test3.ipynb
importing Jupyter notebook from ReplayMemory_test3.ipynb
importing Jupyter notebook from Display_test3.ipynb
importing Jupyter notebook from Map_test3.ipynb
importing Jupyter notebook from BaseDisplay_test3.ipynb
importing Jupyter notebook from Grid_test3.ipynb
importing Jupyter notebook from DeviceManager_test3.ipynb
importing Jupyter notebook from IoTDevice_test3.ipynb
importing Jupyter notebook from State_test3.ipynb
importing Jupyter notebook from StateUtils_test3.ipynb
importing Jupyter notebook from BaseState_test3.ipynb
importing Jupyter notebook from GridActions_test3.ipynb
importing Jupyter notebook from BaseGrid_test3.ipynb
importing Jupyter notebook from ModelStats_test3.ipynb
importing Jupyter notebook from Physics_test3.ipynb
importing Jupyter notebook from GridPhysics_test3.ipynb
importing Jupyter notebook from G

  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
boolean_map_input (InputLayer)  [(None, 63, 63, 4)]  0                                            
__________________________________________________________________________________________________
tf.cast (TFOpLambda)            (None, 63, 63, 4)    0           boolean_map_input[0][0]          
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 21, 21, 4)    0           tf.cast[0][0]                    
__________________________________________________________________________________________________
tf.image.central_crop (TFOpLamb (None, 17, 17, 4)    0           tf.cast[0][0]                    
____________________________________________________________________________________________

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [10:13<00:00,  2.04s/it]


Boundary counter: 0.0
Tour counter: 0.0
Tough counter: 37.0
Successful Landing: 1.0
Current budget: 143.0
